# Auto-Graph-RAG Nations Dataset Demo

This notebook demonstrates Auto-Graph-RAG using the **Nations dataset** from PyKEEN, a real-world knowledge graph of international relations between 14 countries.

## Dataset Overview
- **14 countries**: USA, USSR, China, UK, Brazil, India, Egypt, Israel, Jordan, Indonesia, Cuba, Poland, Burma, Netherlands
- **55 relationship types**: embassy, militaryalliance, economicaid, treaties, tourism, trade, etc.
- **1,592 relationship instances**: Real diplomatic, economic, and political relationships

## What we'll build
A question-answering system that can answer queries like:
- "Which countries have embassies with the USA?"
- "What military alliances exist in the dataset?"
- "Which countries have economic aid relationships?"
- "What are the tourism relationships between countries?"

## Setup and Environment Detection

In [ ]:
!rm auto-graph-rag -R

#Install Dependancies if Running it Colab

In [ ]:
# Uncomment and run if in Colab
# if in_colab:
# Install PyKEEN and Auto-Graph-RAG
!pip install pykeen

# Clone and install auto-graph-rag if not already done

!git clone https://github.com/benjaminwfriedman/auto-graph-rag.git /content/auto-graph-rag
!cd /content/auto-graph-rag && pip install -e .

In [ ]:
import sys
import os
from pathlib import Path

# Detect environment and set working directory
try:
    import google.colab
    in_colab = True
    work_dir = Path("/content/auto-graph-rag/nations_demo_workspace")
    sys.path.append("/content/auto-graph-rag/src")
    print("🔍 Detected Google Colab environment")
except ImportError:
    in_colab = False
    work_dir = Path("./nations_demo_workspace")
    local_src = Path("./src")
    if local_src.exists():
        sys.path.append(str(local_src))
    print("🔍 Detected local environment")

# Create working directory
work_dir.mkdir(exist_ok=True)
print(f"📁 Working directory: {work_dir.absolute()}")

## Environment Variables Setup

In [ ]:
# Set your API keys here
os.environ["OPENAI_API_KEY"] = ""  # Required
os.environ["HF_TOKEN"] = ""      # Optional but recommended

# Check environment
has_openai = bool(os.getenv("OPENAI_API_KEY")) and os.getenv("OPENAI_API_KEY") != "your-openai-api-key-here"
has_hf = bool(os.getenv("HF_TOKEN")) and os.getenv("HF_TOKEN") != "your-huggingface-token-here"

print(f"🔑 OpenAI API Key: {'✅ Available' if has_openai else '❌ Missing - Required for this demo'}")
print(f"🤗 HuggingFace Token: {'✅ Available' if has_hf else '⚠️ Missing - Recommended for model downloads'}")

if not has_openai:
    print("\n❌ Please set your OpenAI API key above to continue with the demo.")
    print("   You can get one at: https://platform.openai.com/api-keys")

## Import Libraries

In [ ]:
import pykeen.datasets
import networkx as nx
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt

# Import Auto-Graph-RAG modules
from auto_graph_rag.modules import (
    GraphBuilder,
    GraphExplorer,
    DataGenerator,
    ModelTrainer,
    QueryExecutor
)

print("✅ All imports successful!")
print("Available modules:")
for module in [GraphBuilder, GraphExplorer, DataGenerator, ModelTrainer, QueryExecutor]:
    print(f"  - {module.__name__}")

## Step 1: Load and Convert Nations Dataset

In [ ]:
def load_nations_dataset():
    """Load the Nations dataset from PyKEEN and convert to NetworkX."""
    print("📦 Loading Nations dataset from PyKEEN...")

    # Load dataset
    dataset = pykeen.datasets.Nations()
    training = dataset.training

    print(f"📊 Dataset Statistics:")
    print(f"  - Countries: {training.num_entities}")
    print(f"  - Relationship types: {training.num_relations}")
    print(f"  - Total relationships: {training.num_triples}")

    # Get mappings
    entity_to_id = training.entity_to_id
    relation_to_id = training.relation_to_id
    id_to_entity = {v: k for k, v in entity_to_id.items()}
    id_to_relation = {v: k for k, v in relation_to_id.items()}

    print(f"\n🌍 Countries in dataset:")
    countries = sorted(entity_to_id.keys())
    for i, country in enumerate(countries):
        if i % 5 == 0 and i > 0:
            print()  # New line every 5 countries
        print(f"{country:12}", end=" ")
    print()

    print(f"\n🔗 Top 10 relationship types:")
    triples = training.mapped_triples
    relation_counts = Counter()
    for i in range(len(triples)):
        h, r, t = triples[i]
        relation = id_to_relation[r.item()]
        relation_counts[relation] += 1

    for relation, count in relation_counts.most_common(10):
        print(f"  - {relation:20}: {count:3d} instances")

    # Create NetworkX graph
    print(f"\n🔄 Converting to NetworkX graph...")
    G = nx.DiGraph()

    # Add nodes (countries) with metadata
    for country in entity_to_id.keys():
        G.add_node(country, type="Country", name=country, region="Unknown")

    # Add edges (relationships)
    for i in range(len(triples)):
        h, r, t = triples[i]
        head_country = id_to_entity[h.item()]
        relation = id_to_relation[r.item()]
        tail_country = id_to_entity[t.item()]

        # Add edge with relation type
        G.add_edge(head_country, tail_country, type=relation, relation=relation)

    print(f"✅ NetworkX graph created:")
    print(f"  - Nodes: {G.number_of_nodes()}")
    print(f"  - Edges: {G.number_of_edges()}")
    print(f"  - Average degree: {sum(dict(G.degree()).values()) / G.number_of_nodes():.1f}")

    return G, relation_counts

# Load the dataset
nations_graph, relation_counts = load_nations_dataset()

## Dataset Visualization

In [ ]:
# Visualize relationship distribution
plt.figure(figsize=(12, 6))
top_relations = dict(relation_counts.most_common(15))
plt.bar(range(len(top_relations)), list(top_relations.values()))
plt.xticks(range(len(top_relations)), list(top_relations.keys()), rotation=45, ha='right')
plt.title('Top 15 International Relationship Types in Nations Dataset')
plt.ylabel('Number of Instances')
plt.tight_layout()
plt.show()

# Show some example relationships
print("\n💡 Example relationships:")
edges = list(nations_graph.edges(data=True))
import random
random.seed(42)
sample_edges = random.sample(edges, min(10, len(edges)))

for source, target, data in sample_edges:
    relation = data['relation']
    print(f"  {source:12} --[{relation:15}]--> {target}")

## Step 2: Build Graph Database with GraphBuilder

In [ ]:
# Initialize GraphBuilder
builder = GraphBuilder()

print("🔧 GraphBuilder Info:")
info = builder.get_info()
for key, value in info.items():
    print(f"  {key}: {value}")

print("\n📦 Building Nations graph database...")

# Extract node and edge labels
node_labels = {node: data["type"] for node, data in nations_graph.nodes(data=True)}
edge_labels = {(u, v): data["type"] for u, v, data in nations_graph.edges(data=True)}

print(f"🏷️ Creating labels:")
print(f"  - Node types: {set(node_labels.values())}")
print(f"  - Edge types: {len(set(edge_labels.values()))} unique relationship types")

# Build the database
db_path = work_dir / "nations_db"
stats = builder.build_from_networkx(
    graph=nations_graph,
    db_path=db_path,
    graph_name="nations",
    node_labels=node_labels,
    edge_labels=edge_labels
)

print("\n✅ Nations graph database created!")
print(f"📊 Database Statistics:")
for key, value in stats.items():
    if isinstance(value, list) and len(value) > 5:
        print(f"  {key}: {len(value)} items (showing first 5: {value[:5]}...)")
    else:
        print(f"  {key}: {value}")

## Step 3: Explore Schema with GraphExplorer

In [ ]:
!cd auto-graph-rag/ && git branch

In [ ]:
# Initialize GraphExplorer
if not has_openai:
    print("❌ OpenAI API key required for schema exploration. Skipping this step.")
else:
    explorer = GraphExplorer(llm_provider="openai", llm_model="gpt-4")

    print("🔧 GraphExplorer Info:")
    info = explorer.get_info()
    for key, value in info.items():
        print(f"  {key}: {value}")

    print("\n🕵️ Exploring Nations graph schema...")
    print("This will analyze the international relationships using GPT-4...")

    # Explore the database
    schema_path = work_dir / "nations_schema.json"
    schema = explorer.explore_from_db(
        db_path=db_path,
        max_samples=20,  # Sample more for richer analysis
        save_schema_to=schema_path
    )

    print("\n✅ Schema exploration complete!")

    # Display schema results
    print(f"\n📋 Discovered Schema Summary:")
    print(f"  {schema['summary']}")

    print(f"\n🌍 Node Types: {list(schema['nodes'].keys())}")
    print(f"🔗 Edge Types: {len(list(schema['edges'].keys()))} relationship types discovered")

    # Show details for Country node type
    if 'Country' in schema['nodes']:
        country_info = schema['nodes']['Country']
        print(f"\n🏛️ Country Node Details:")
        print(f"  Description: {country_info['description']}")
        print(f"  Properties: {country_info['properties']}")
        print(f"  Example: {country_info['example_values']}")

    # Show some interesting relationship types
    print(f"\n🔗 Sample International Relationships:")
    for i, (rel_type, rel_info) in enumerate(list(schema['edges'].items())[:8]):
        print(f"  {i+1}. {rel_type}: {rel_info['description']}")

## Step 4: Generate Training Data with DataGenerator

In [ ]:
# Initialize DataGenerator
if not has_openai:
    print("❌ OpenAI API key required for training data generation. Skipping this step.")
else:
    generator = DataGenerator(llm_provider="openai", llm_model="gpt-4")

    print("🔧 DataGenerator Info:")
    info = generator.get_info()
    for key, value in info.items():
        print(f"  {key}: {value}")

    print("\n📝 Generating training data for international relations...")
    print("This will create diverse question-Cypher pairs about countries and their relationships...")

    # Generate dataset from the schema
    dataset_path = work_dir / "nations_dataset.jsonl"
    dataset = generator.generate_from_schema(
        schema_path=schema_path,
        num_examples=150,  # Good balance for demo
        output_path=dataset_path,
        complexity_distribution={
            1: 0.25,  # Simple lookups: "List all countries", "What countries exist?"
            2: 0.25,  # Filtered queries: "Countries with embassies", "Military alliances"
            3: 0.25,  # Relationships: "Which countries have diplomatic relations?"
            4: 0.25,  # Aggregations: "Count of relationships by type"
        },
        db_path=db_path  # For query validation
    )

    print("\n✅ Training data generated!")

    # Analyze the dataset
    print(f"\n📊 Dataset Statistics:")
    print(f"  Total examples: {len(dataset)}")

    # Complexity and intent analysis
    complexity_counts = Counter(item.get('complexity', 0) for item in dataset)
    intent_counts = Counter(item.get('intent', 'unknown') for item in dataset)

    print(f"  Complexity distribution: {dict(complexity_counts)}")
    print(f"  Intent types: {len(intent_counts)} different intents")

    # Show sample examples focused on international relations
    print(f"\n💡 Sample Training Examples for International Relations:")

    # Find examples with interesting relations
    interesting_examples = []
    keywords = ['embassy', 'military', 'economic', 'alliance', 'tourism', 'trade', 'diplomatic']

    for example in dataset:
        question = example.get('question', '').lower()
        cypher = example.get('cypher', '').lower()
        if any(keyword in question or keyword in cypher for keyword in keywords):
            interesting_examples.append(example)

    # Show a mix of examples
    examples_to_show = interesting_examples[:3] + dataset[:2]  # 3 interesting + 2 random

    for i, example in enumerate(examples_to_show[:5], 1):
        print(f"\n  Example {i}:")
        print(f"    Question: {example.get('question', 'N/A')}")
        cypher = example.get('cypher', 'N/A')
        if len(cypher) > 80:
            cypher = cypher[:77] + "..."
        print(f"    Cypher: {cypher}")
        print(f"    Complexity: {example.get('complexity', 'N/A')}")
        print(f"    Intent: {example.get('intent', 'N/A')}")

## Step 5: Fine-tune Model with ModelTrainer

⚠️ **Note**: Model training is computationally intensive. For this demo, we'll use a small, efficient configuration.

In [ ]:
# Initialize ModelTrainer
trainer = ModelTrainer()

print("🔧 ModelTrainer Info:")
info = trainer.get_info()
for key, value in info.items():
    print(f"  {key}: {value}")

# Configuration for efficient training
model_config = {
    "base_model": "meta-llama/Llama-3.2-1B-Instruct",  # Small, efficient model
    "epochs": 15, # Reduced for demo
    "learning_rate": 5e-4,
    "batch_size": 4,
    "lora_rank": 8,
    "output_dir": str(work_dir / "nations_model")
}

print(f"\n⚙️ Training Configuration:")
for key, value in model_config.items():
    print(f"  {key}: {value}")

print(f"\n🎯 Starting model training for international relations...")
print(f"Training a specialized model to understand diplomatic, economic, and political relationships!")

# Train the model
model = trainer.train_from_file(
    dataset_path=dataset_path,
    model_name=model_config["base_model"],
    output_dir=Path(model_config["output_dir"]),
    epochs=model_config["epochs"],
    learning_rate=model_config["learning_rate"],
    batch_size=model_config["batch_size"],
    lora_rank=model_config["lora_rank"]
)

print("\n✅ Model training completed!")
print(f"🎉 Nations-specialized model ready for international relations queries!")

## Step 6: Query the International Relations Graph with QueryExecutor

In [ ]:
# Initialize QueryExecutor
executor = QueryExecutor()

print("🔧 QueryExecutor Info:")
info = executor.get_info()
for key, value in info.items():
    print(f"  {key}: {value}")

# Use the trained model and database
model_path = work_dir / "nations_model"
print(f"\n🎯 Using Assets:")
print(f"  Model: {model_path} (Nations-specialized model)")
print(f"  Database: {db_path} (Nations knowledge graph)")

# International relations test questions
test_questions = [
    "What countries are in the dataset?",
    "Which countries have embassy relationships?",
    "What treaties exist between countries?",
    "What countries gained independance from china?",
    "Which country is in the most ngo relatinoships with other countries?",
    "Which countries does the USA have violent actions with?"
]

print(f"\n❓ Test Questions for International Relations:")
for i, question in enumerate(test_questions, 1):
    print(f"  {i}. {question}")

print(f"\n🤖 Query Execution with Nations-Specialized Model:")
print(f"Let's see how our model handles international relations queries!")

for i, question in enumerate(test_questions, 1):
    print(f"\n{'='*60}")
    print(f"Query {i}: {question}")
    print(f"{'='*60}")

    result = executor.query_with_model(
        question=question,
        model_path=model_path,
        db_path=db_path,
        return_cypher=True,
        format_results=True
    )

    if result['success']:
        print(f"✅ Generated Cypher: {result['cypher']}")
        print(f"📊 Results: {result['count']} rows")

        if 'results' in result and result['results']:
            print(f"\n🔍 Sample Results:")
            # Show more results for international relations
            max_results = min(8, len(result['results']))
            for j, row in enumerate(result['results'][:max_results]):
                print(f"    {j+1:2d}. {row}")
            if len(result['results']) > max_results:
                print(f"    ... and {len(result['results']) - max_results} more")
    else:
        print(f"❌ Error: {result['error']}")

    print()  # Add space between queries